In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q snorkel spacy-nightly[transformers,cuda100] --pre

     |████████████████████████████████| 369.7MB 45kB/s 
     |████████████████████████████████| 122kB 50.3MB/s 
     |████████████████████████████████| 1.0MB 32.5MB/s 
     |████████████████████████████████| 2.1MB 29.7MB/s 
     |████████████████████████████████| 12.8MB 31.5MB/s 
     |████████████████████████████████| 901kB 38.4MB/s 
     |████████████████████████████████| 3.3MB 25.6MB/s 
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import string
import spacy
import spacy_transformers
import pandas as pd

from spacy.cli import download
download("en_core_web_trf")

nlp = spacy.load("en_core_web_trf")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


/usr/local/lib/python3.7/dist-packages/spacy/util.py:693: UserWarning: [W095] Model 'en_core_web_trf' (3.0.0) requires spaCy >=3.0.0,<3.1.0 and is incompatible with the current version (3.0.0rc5). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
!pip install -q sent2vec

In [ ]:
from scipy import spatial
from sent2vec.vectorizer import Vectorizer
vectorizer = Vectorizer()

# Part I Check the answer is relevant to the question or not 

Some approaches I have been researching:
> **1 Intent Recognition.** We need to label the intent of the answer based on the intent of the question which we already have. To detect irrelevant answers we need a `irrelevant` label. i.e:
>> Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities? -> `self_improvement` <br>
A: Yes, I do. I have improved my technical skill as well as my ability of team working. -> `self_improvement` <br>
A: Yes, I do. I play basketball very well ->  `irrelevant`

> This is classification problem, a suppervised learning, but we don't have any data so we can't try now, but it's worth trying when we collect enough data!

> **2.Intent Recognition combine Entity Recognition** Entity Recognition makes the intent more specific. i.e:
>> Q: In the last year, do you feel you have grown in terms of your `knowledge` and `capabilities`  <br> -> `{intent`:`self_improvement, entites: {knowledge, capabilities}}`

> As mentioned above, this is classification problem for both Intent Recognition and Entity Recognition, a suppervised learning, but we don't have any data so I can't try now, but it's worth trying when we collect enough data!

>**3. Sentence Similarity**: In this method, I'll calculate the distance (`cosine distance`) between the question and the answer. The intuitive of this is: the closer distance the more similar between question and answer (more relevant). To calculate `cosine distance` I need embed the sentence into vector. To do this, I have three options:
 - I clean the sentence, just keep main words then use pre-trained model Word2Vec to embedding each word in the sentence then calculate the similarity of all posible pairs of words (one in the answer one in the question) then get the average
 - I use pre-trained model sent2vec library based on Word2Vec and BERT this is a unsupervised powerful model to embedding semantic of words. <br>
 - Another option is using pre-trained BERT to embedding the sentence. <br>

>After calculating distance, now the question is: which threshold to define relevant or irrelevant? The threshold can be dynamic for sequential data, If we continuously have the label of data then we can update the threshold by statistic formula. This threshold will be the lower bound of outlier. Another approach is that solve the problem by binary classification (when we have data)

>**4. Classification - Fine-tuning pre-trained BERT**: In this method I will use the question-answering architecture of BERT to represent the question-answer pairs. Then I add a binary classification layer to learn relevant or irrelevant. 

**In this test, I use the third approach due to lack of data. But I can take advantage of powerful pre-trained model**

## Calculate similarity word-pairs using pre-trained Word2Vec

Using **Spacy** library for text processing 

In [ ]:
# Define a function to normalize text:
def text_processing(text):
  # lowercase text
  text = text.lower()
  doc = nlp(text)
  # remove punctuation
  doc = [token for token in doc if token.text not in string.punctuation]
  # remove stop words 
  doc = [token for token in doc if not token.is_stop]
  # get lemma 
  doc = [token.lemma_ for token in doc]

  return doc

Using **pre-trained Word2Vec** for word embedding

In [ ]:
import gensim
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
sentences = [
    "In the last year, do you feel you have grown in terms of your knowledge and capabilities?",
    "Yes, I do. I have improved my technical skill.",
    "I play basketball very well.",
    "Yes, I do. I have improved my technical skill as well as my ability of working in a team",
    "I learned a lot in my school, I worked very hard",
    "You are so bad.",
    "I almost learned nothing in the last year, the job is unsuitable with my career path",
    "I have learned new techniques and improved my communication skill",
    "Everything seems boring, I did not learn anything",
]

cleaned_sentences = []
for i, sent in enumerate(sentences):
  cleaned_sentences.append(text_processing(sent))

def calculate_similarity(doc_1, doc_2):
  sim = 0
  for token_1 in doc_1:
    for token_2 in doc_2:
      sim += spatial.distance.cosine(glove_vectors[token_1], glove_vectors[token_2])         #glove_vectors.similarity(token_1, token_2)
  if len(doc_2) > 0:
    sim /= (len(doc_1)*len(doc_2))
    return sim
  return 0

for i,sent in enumerate(sentences[1:]):
  print('Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?')
  print('A:',sentences[i+1])
  print('Distance score:', calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')

Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?
A: Yes, I do. I have improved my technical skill.
Distance score: 0.7346890200860798
---------------
Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?
A: I play basketball very well.
Distance score: 0.801333483386164
---------------
Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?
A: Yes, I do. I have improved my technical skill as well as my ability of working in a team
Distance score: 0.7133609318130073
---------------
Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?
A: I learned a lot in my school, I worked very hard
Distance score: 0.6978753939270973
---------------
Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?
A: You are so bad.
Distance score: 0.7320054955780506
---------------
Q: In the last year, do you feel you

> We can see that, this approach is not good. The answer "I learned a lot in my school, I worked very hard" quiet not related to the question but have the best score (lowest). The two expected answer don't have good score (quiet high)

## Using Sent2Vec based on pre-trained Word2Vec

I use 0.65 as threshold for both questions

**Question 1** In the last year, do you feel you have grown in terms of your knowledge and capabilities?



In [ ]:

sentences = [
    "In the last year, do you feel you have grown in terms of your knowledge and capabilities?",
    "Yes, I do. I have improved my technical skill.",
    "I play basketball very well.",
    "Yes, I do. I have improved my technical skill as well as my ability of working in a team",
    "I learned a lot in my school, I worked very hard",
    "You are so bad.",
    "I almost learned nothing in the last year, the job is unsuitable with my career path",
    "I have learned new techniques and improved my communication skill",
    "Everything seems boring, I did not learn anything",
]
tokens_sent = []
relevant_sents = []
irrelevant_sents = []

nlp.vocab["not"].is_stop = False
for sent in sentences:
  tokens_sent.append(text_processing(sent))

vectorizer.word2vec(tokens_sent, pretrained_vectors_path= '/content/drive/MyDrive/lexvec.commoncrawl.300d.W.pos.neg3.vectors')
vectors = vectorizer.vectors

for i,sent in enumerate(sentences[1:]):
  print(sentences[i+1])
  if spatial.distance.cosine(vectors[0], vectors[i+1]) < 0.65:
    relevant_sents.append(sentences[i+1])
  else:
    irrelevant_sents.append(sentences[i+1])
  print('Distance score:', spatial.distance.cosine(vectors[0], vectors[i+1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')

print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)

Yes, I do. I have improved my technical skill.
Distance score: 0.4172677993774414
---------------
I play basketball very well.
Distance score: 0.8272860050201416
---------------
Yes, I do. I have improved my technical skill as well as my ability of working in a team
Distance score: 0.35876572132110596
---------------
I learned a lot in my school, I worked very hard
Distance score: 0.4818311333656311
---------------
You are so bad.
Distance score: 0.7784879803657532
---------------
I almost learned nothing in the last year, the job is unsuitable with my career path
Distance score: 0.4150501489639282
---------------
I have learned new techniques and improved my communication skill
Distance score: 0.42051494121551514
---------------
Everything seems boring, I did not learn anything
Distance score: 0.5333107113838196
---------------

Summary

Relevant answers:

	- Yes, I do. I have improved my technical skill.
	- Yes, I do. I have improved my technical skill as well as my ability of workin

> One sentence: "I learned a lot in my school, I worked very hard" is wrongly classified

**Question. 2** Do you feel the organization's policies and benefits are employee-friendly?,


In [ ]:
sentences = ["Do you feel the organization's policies and benefits are employee-friendly?",
             "I feel the policy of the company is quite good, brings good conditions for employees!",
            "The company does not have a policy to support employees away from home. Should facilitate working from home or remotely",
            "Overtime payment is quite low, I'm not satisfied about this",
            "Hi, how are you",
            "Just fine, but the company should have more outside activities",
            "They are so nice to me!",
            "I have to work early tomorrow",
            "Everything seems boring, I did not learn anything",
            "I don't know",
            "The policies and benefits are good!"
]
tokens_sent = []
relevant_sents = []
irrelevant_sents = []

nlp.vocab["not"].is_stop = False
for sent in sentences:
  tokens_sent.append(text_processing(sent))

vectorizer.word2vec(tokens_sent, pretrained_vectors_path= '/content/drive/MyDrive/lexvec.commoncrawl.300d.W.pos.neg3.vectors')
vectors = vectorizer.vectors
for i,sent in enumerate(sentences[1:]):
  #print('Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?')
  print(sentences[i+1])
  if spatial.distance.cosine(vectors[0], vectors[i+1]) < 0.65:
    relevant_sents.append(sentences[i+1])
  else:
    irrelevant_sents.append(sentences[i+1])
  print('Distance score:', spatial.distance.cosine(vectors[0], vectors[i+1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')

print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)

I feel the policy of the company is quite good, brings good conditions for employees!
Distance score: 0.23459845781326294
---------------
The company does not have a policy to support employees away from home. Should facilitate working from home or remotely
Distance score: 0.273262083530426
---------------
Overtime payment is quite low, I'm not satisfied about this
Distance score: 0.55739426612854
---------------
Hi, how are you
Distance score: 0.9538920782506466
---------------
Just fine, but the company should have more outside activities
Distance score: 0.5377933084964752
---------------
They are so nice to me!
Distance score: 0.7342964112758636
---------------
I have to work early tomorrow
Distance score: 0.7378312945365906
---------------
Everything seems boring, I did not learn anything
Distance score: 0.6842125952243805
---------------
I don't know
Distance score: 0.7175776660442352
---------------
The policies and benefits are good!
Distance score: 0.22672039270401
------------

> All sentences are classified correctly!

## Using Sent2Vec based on pre-trained BERT

I use 0.1 as threshold for both question

**Question 1** In the last year, do you feel you have grown in terms of your knowledge and capabilities?



In [ ]:
sentences = [
    "In the last year, do you feel you have grown in terms of your knowledge and capabilities?",
    "Yes, I do. I have improved my technical skill.",
    "I play basketball very well.",
    "Yes, I do. I have improved my technical skill as well as my ability of working in a team",
    "I learned a lot in my school, I worked very hard",
    "You are so bad.",
    "I almost learned nothing in the last year, the job is unsuitable with my career path",
    "I have learned new techniques and improved my communication skill",
    "Everything seems boring, I did not learn anything",
]

relevant_sents = []
irrelevant_sents = []
for i,sent in enumerate(sentences[1:]):
  vectorizer.bert([sentences[0],sent])
  vectors_bert = vectorizer.vectors
#  print('Q:',sentences[0])
  print(sentences[i+1])
  if  spatial.distance.cosine(vectors_bert[0], vectors_bert[1]) < 0.1:
#    print('Relevant')
    relevant_sents.append(sent)
  else:
    irrelevant_sents.append(sent)
 #   print('Irrelevant')
  print('Distance score:', spatial.distance.cosine(vectors_bert[0], vectors_bert[1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')
print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)


Yes, I do. I have improved my technical skill.
Distance score: 0.06629043817520142
---------------
I play basketball very well.
Distance score: 0.1992979645729065
---------------
Yes, I do. I have improved my technical skill as well as my ability of working in a team
Distance score: 0.0367618203163147
---------------
I learned a lot in my school, I worked very hard
Distance score: 0.15720242261886597
---------------
You are so bad.
Distance score: 0.226995050907135
---------------
I almost learned nothing in the last year, the job is unsuitable with my career path
Distance score: 0.059124886989593506
---------------
I have learned new techniques and improved my communication skill
Distance score: 0.08694076538085938
---------------
Everything seems boring, I did not learn anything
Distance score: 0.15345317125320435
---------------

Summary

Relevant answers:

	- Yes, I do. I have improved my technical skill.
	- Yes, I do. I have improved my technical skill as well as my ability of wor

> One sentence "Everything seems boring, I did not learn anything" is wrongly classified

**Question. 2** Do you feel the organization's policies and benefits are employee-friendly?,


In [ ]:
sentences = ["Do you feel the organization's policies and benefits are employee-friendly?",
            "I feel the policy of the company is quite good, brings good conditions for employees!",
            "The company does not have a policy to support employees away from home. Should facilitate working from home or remotely",
            "Overtime payment is quite low, I'm not satisfied about this",
            "Hi, how are you",
            "Just fine, but the company should have more outside activities",
            "They are so nice to me!",
            "I have to work early tomorrow",
            "Everything seems boring, I did not learn anything",
            "I don't know",
            "The policies and benefits are good!"
]

relevant_sents = []
irrelevant_sents = []
for i,sent in enumerate(sentences[1:]):
  vectorizer.bert([sentences[0],sent])
  vectors_bert = vectorizer.vectors
  #print('Q:',sentences[0])
  print(sentences[i+1])
  if  spatial.distance.cosine(vectors_bert[0], vectors_bert[1]) < 0.1:
  #  print('Relevant')
    relevant_sents.append(sent)
  else:
    irrelevant_sents.append(sent)
  #  print('Irrelevant')
  print('Distance score:', spatial.distance.cosine(vectors_bert[0], vectors_bert[1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')
print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)

I feel the policy of the company is quite good, brings good conditions for employees!
Distance score: 0.0843970775604248
---------------
The company does not have a policy to support employees away from home. Should facilitate working from home or remotely
Distance score: 0.09428781270980835
---------------
Overtime payment is quite low, I'm not satisfied about this
Distance score: 0.05280721187591553
---------------
Hi, how are you
Distance score: 0.19934695959091187
---------------
Just fine, but the company should have more outside activities
Distance score: 0.057079195976257324
---------------
They are so nice to me!
Distance score: 0.1057860255241394
---------------
I have to work early tomorrow
Distance score: 0.11201858520507812
---------------
Everything seems boring, I did not learn anything
Distance score: 0.1070060133934021
---------------
I don't know
Distance score: 0.11967694759368896
---------------
The policies and benefits are good!
Distance score: 0.11891794204711914


> One sentence "The policies and benefits are good!" is wrongly classified 

## Using pre-trained BERT



In [ ]:
pip install -q sentence_transformers 

     |████████████████████████████████| 81kB 5.5MB/s 
     |████████████████████████████████| 1.2MB 11.3MB/s 


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('nli-distilroberta-base-v2')

I use 0.6 as threshold for both questions

**Question 1.** In the last year, do you feel you have grown in terms of your knowledge and capabilities?

In [ ]:
sentences = [
    "In the last year, do you feel you have grown in terms of your knowledge and capabilities?",
    "Yes, I do. I have improved my technical skill.",
    "I play basketball very well.",
    "Yes, I do. I have improved my technical skill as well as my ability of working in a team",
    "I learned a lot in my school, I worked very hard",
    "You are so bad.",
    "I almost learned nothing in the last year, the job is unsuitable with my career path",
    "I have learned new techniques and improved my communication skill",
    "Everything seems boring, I did not learn anything",
]

sentence_embeddings = model.encode(sentences)
relevant_sents = []
irrelevant_sents = []
for i,sent in enumerate(sentences[1:]):
  #print('Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?')
  print(sentences[i+1])
  if  spatial.distance.cosine(sentence_embeddings[0], sentence_embeddings[i+1]) < 0.6:
   # print('Relevant')
    relevant_sents.append(sent)
  else:
   # print('Irrelevant')
    irrelevant_sents.append(sent)
  print('Distance score:', spatial.distance.cosine(sentence_embeddings[0], sentence_embeddings[i+1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')

print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)

Yes, I do. I have improved my technical skill.
Distance score: 0.5134833753108978
---------------
I play basketball very well.
Distance score: 0.8135423809289932
---------------
Yes, I do. I have improved my technical skill as well as my ability of working in a team
Distance score: 0.5630861520767212
---------------
I learned a lot in my school, I worked very hard
Distance score: 0.670462429523468
---------------
You are so bad.
Distance score: 0.8805501386523247
---------------
I almost learned nothing in the last year, the job is unsuitable with my career path
Distance score: 0.7736584097146988
---------------
I have learned new techniques and improved my communication skill
Distance score: 0.5645861327648163
---------------
Everything seems boring, I did not learn anything
Distance score: 1.0033836120273918
---------------

Summary

Relevant answers:

	- Yes, I do. I have improved my technical skill.
	- Yes, I do. I have improved my technical skill as well as my ability of working i

> Two sentences: "I almost learned nothing in the last year, the job is unsuitable with my career path" and "Everything seems boring, I did not learn anything" are wrongly classified

**Question 2.**Do you feel the organization's policies and benefits are employee-friendly?

In [ ]:
sentences = [
   "Do you feel the organization's policies and benefits are employee-friendly?",
    "I feel the policy of company is quiet good, brings good conditions for employees!",
    "The company does not have a policy to support employees away from home. Should facilitate working from home or remotely",
    "Overtime payment is quite low, I'm not satisfied about this",
    "Hi, how are you",
    "Just fine, but the company should have more outside activities",
    "They are so nice to me!",
    "I have to work early tomorrow",
    "Everything seems boring, I did not learn anything",
    "I don't know",
    "The policies and benefits are good!"
]

sentence_embeddings = model.encode(sentences)
relevant_sents = []
irrelevant_sents = []
for i,sent in enumerate(sentences[1:]):
 # print('Q: In the last year, do you feel you have grown in terms of your knowledge and capabilities?')
  print(sentences[i+1])
  if  spatial.distance.cosine(sentence_embeddings[0], sentence_embeddings[i+1]) < 0.6:
  #  print('Relevant')
    relevant_sents.append(sent)
  else:
  #  print('Irrelevant')
    irrelevant_sents.append(sent)
  print('Distance score:', spatial.distance.cosine(sentence_embeddings[0], sentence_embeddings[i+1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
  print('---------------')

print('\nSummary\n')
print('Relevant answers:\n')
for sent in relevant_sents:
  print('\t-',sent)

print('\nIrrelevant answers:\n')
for sent in irrelevant_sents:
  print('\t-',sent)

I feel the policy of company is quiet good, brings good conditions for employees!
Distance score: 0.34289824962615967
---------------
The company does not have a policy to support employees away from home. Should facilitate working from home or remotely
Distance score: 0.4734971523284912
---------------
Overtime payment is quite low, I'm not satisfied about this
Distance score: 0.7398489415645599
---------------
Hi, how are you
Distance score: 0.6443432569503784
---------------
Just fine, but the company should have more outside activities
Distance score: 0.45370060205459595
---------------
They are so nice to me!
Distance score: 0.7259829044342041
---------------
I have to work early tomorrow
Distance score: 0.8282761871814728
---------------
Everything seems boring, I did not learn anything
Distance score: 0.9315886124968529
---------------
I don't know
Distance score: 0.9296606630086899
---------------
The policies and benefits are good!
Distance score: 0.45825880765914917
---------

> One sentence: "Overtime payment is quite low, I'm not satisfied about this" is wrongly classified

# Part II Sentiment analysis

> After having the answers, we classify the answers into three groups: `positive`, `negative` or `neutral` 

> The problem can be solved easily when we have dataset. But in the case we don't have any. I propose a statistic method:

>> I use two dictionaries (LoughranMcDonald and Twitter) including: positive and negative words. Then I count the number of negative and positive words in the answer to calculate polarity score


In [ ]:
LoughranMcDonald_negative = pd.read_excel('/content/drive/MyDrive/LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name=1, header=None)
LoughranMcDonald_negative = LoughranMcDonald_negative.values.flatten()

LoughranMcDonald_positive = pd.read_excel('/content/drive/MyDrive/LoughranMcDonald_SentimentWordLists_2018.xlsx', sheet_name=2, header=None)
LoughranMcDonald_positive = LoughranMcDonald_positive.values.flatten()

f = open('/content/drive/MyDrive/twitter-negative-words.txt',encoding="ISO-8859-1")
twitter_negative = f.read()
twitter_negative = twitter_negative.split('\n')

f = open('/content/drive/MyDrive/twitter-positive-words.txt',encoding="ISO-8859-1")
twitter_positive = f.read()
twitter_positive = twitter_positive.split('\n')

positive_words = [item.lower() for item in LoughranMcDonald_positive]
negative_words = [item.lower() for item in LoughranMcDonald_negative]

for item in twitter_positive:
  if item not in positive_words:
    positive_words.append(item)

for item in twitter_negative:
  if item not in negative_words:
    negative_words.append(item)

print('Number of Negative words:', len(negative_words))
print('Number of Positive words:', len(positive_words))

Number of Negative words: 6298
Number of Positive words: 2169


In [ ]:
def calculate_polarity(doc):
  neg_score = 0
  pos_score = 0

  for token in doc:
    if token.text.lower() in negative_words:
      neg_score += 1
    if token.text.lower() in positive_words:
      pos_score += 1
  polarity_score = (pos_score-neg_score)/((pos_score+neg_score)+0.0000001)

  return polarity_score
    

In [ ]:
A1= [
    "Yes, I do. I have improved my technical skill.",
    "I play basketball very well.",
    "Yes, I do. I have improved my technical skill as well as my ability of working in a team",
    "I learned a lot in my school, I worked very hard",
    "You are so bad.",
    "I almost learned nothing in the last year, the job is unsuitable with my career path",
    "I have learned new techniques and improved my communication skill",
    "Everything seems boring, I did not learn anything",
    "Have improved more than before, being exposed to many practical jobs, applying learned knowledge into practice, solving many problems at work",
    "It's quite regretful and time-consuming because I did not work according to my passion, so I could not consolidate any more knowledge",
    "I feel the policy of company is quite good, brings good conditions for employees!",
    "The company does not have a policy to support employees away from home. Should facilitate working from home or remotely",
    "Overtime payment is quite low, I'm not satisfied about this",
    "Hi, how are you",
    "Just fine, but the company should have more outside activities",
    "They are so nice to me!",
    "I have to work early tomorrow",
    "I don't know",
    "The policies and benefits are good!"
]

positive_sents = []
negative_sents = []
neutral_sents = []
for sent in A1:
  doc = nlp(sent)
  polarity_score = calculate_polarity(doc)
  if polarity_score < 0: 
    negative_sents.append(sent)
  elif polarity_score > 0:
    positive_sents.append(sent)
  else:
    neutral_sents.append(sent)

print('\nSummary\n')
print('Positive:\n')

for sent in positive_sents:
  print('\t-',sent)

print('\nNegative:\n')
for sent in negative_sents:
  print('\t-',sent)

print('\nNeutral:\n')
for sent in neutral_sents:
  print('\t-',sent)


Summary

Positive:

	- Yes, I do. I have improved my technical skill.
	- I play basketball very well.
	- Yes, I do. I have improved my technical skill as well as my ability of working in a team
	- I have learned new techniques and improved my communication skill
	- Have improved more than before, being exposed to many practical jobs, applying learned knowledge into practice, solving many problems at work
	- It's quite regretful and time-consuming because I did not work according to my passion, so I could not consolidate any more knowledge
	- I feel the policy of company is quite good, brings good conditions for employees!
	- The company does not have a policy to support employees away from home. Should facilitate working from home or remotely
	- Overtime payment is quite low, I'm not satisfied about this
	- Just fine, but the company should have more outside activities
	- They are so nice to me!
	- I have to work early tomorrow
	- The policies and benefits are good!

Negative:

	- You

> Not bad result!

**Create user interaction (for later)**

In [ ]:
exit_list = ['exit', 'see you later', 'bye', 'quit', 'break']
print('Bot: Hi, I have some questions for you!',)
print("In the last year, do you feel you have grown in terms of your knowledge and capabilities?",)
while(True):
  user_input = input()
  if user_input.lower() in exit_list:
    print('Bot: Chat with you later!')
    break
  else:
    sentences = [
     "In the last year, do you feel you have grown in terms of your knowledge and capabilities?",
     user_input
    ]
    vectorizer.bert(sentences)
    vectors_bert = vectorizer.vectors
    print('Distance score:', spatial.distance.cosine(vectors_bert[0], vectors_bert[1]))# calculate_similarity(cleaned_sentences[0], cleaned_sentences[i+1]))
    

# Conclusion and Further work

> **sent2vec based on pretrained BERT and Word2Vec** give best results. We can use this method to label data <br>
> **sentiment analysis based on statistic** give a baseline, because it count on word-base, i.e: it can't distinguish between `not sastified` and `sastified` 

> **Furthur work**: We can implements the approaches which I have mentioned on each part when we collect and label data manually.  